In [ ]:
# Load MNIST data set

from tensorflow.keras.datasets import mnist
train_set, test_set = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# generate a simple NN model

from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np

    
kernel_initializer = 'he_normal'
activation = "relu"

def get_model(chs=128):
    shape=(28,28,1)
    
    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs*2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model



In [ ]:
!pip install dahuffman

In [ ]:
!pip install joblib

In [ ]:
import dahuffman
import pickle
import joblib
import base64


In [ ]:
import numpy as np
import dahuffman
from tensorflow import keras

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        
    def call(self, inputs):
        return self.net_model(inputs)
    
    def compressNN(self):
        # Get the original weights
        weights = self.net_model.get_weights()

        # Convert the weights to a flat array
        flattened_weights = np.concatenate([w.flatten() for w in weights])

        # Compress the weights using Huffman encoding
        codec = dahuffman.HuffmanCodec.from_data(flattened_weights)
        compressed_weights = codec.encode(flattened_weights)

        # Convert the compressed weights to bytes
        compressed_bytes = compressed_weights

        return compressed_bytes

    def decompressNN(self, compressed_weights):
        # Instantiate the Huffman codec
        codec = dahuffman.HuffmanCodec.from_binary(compressed_weights)

        # Decode the compressed weights using Huffman decoding
        deserialized_weights = codec.decode(compressed_weights)
        
        
        print(deserialized_weights)
#         # Reshape the weights to match the original shapes
#         shapes = [w.shape for w in self.net_model.get_weights()]
#         reshaped_weights = []
#         idx = 0
#         for shape in shapes:
#             weight_size = np.prod(shape)
#             weight_array = np.array(deserialized_weights[idx:idx+weight_size])
#             if weight_array.size == 1 and weight_size > 1:
#                 # Broadcast the single value to match the target shape
#                 weight_array = np.full(shape, np.squeeze(weight_array))
#             elif weight_array.size != weight_size:
#                 raise ValueError(f"Cannot reshape array of size {weight_array.size} into shape {shape}")
#             reshaped_weight = np.reshape(weight_array, shape)
#             reshaped_weights.append(reshaped_weight)
#             idx += weight_size
        
#         print(type(reshaped_weights))
        
        # Set the decompressed weights to the network
#         self.net_model.set_weights(reshaped_weights)

        return self.net_model


    def compare_weights(self, original_weights, decompressed_weights):
        differences = []
        for orig, decomp in zip(original_weights, decompressed_weights):
            diff = np.abs(orig - decomp)
            differences.append(diff)
        return differences

# Instantiate the CompressibleNN class and create the original model
model = get_model()  # Replace with your own model creation code
compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = np.random.rand(10, 28, 28, 1)

# Get the original weights
original_weights = model.get_weights()

# Compress the weights
compressed_weights = compNN.compressNN()

# Decompress the weights
decompressed_model = compNN.decompressNN(compressed_weights)
decompressed_weights = decompressed_model.get_weights()

# Compare the original weights with the decompressed weights
differences = compNN.compare_weights(original_weights, decompressed_weights)

# Print the maximum difference for each weight matrix
for i, diff in enumerate(differences):
    print(f"Weight matrix {i}: Max difference = {np.max(diff)}")


In [1]:
workedfrom tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman

kernel_initializer = 'he_normal'
activation = "relu"


def get_model(chs=128):
    shape = (28, 28, 1)

    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs * 2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.codec = []

    def compressNN(self, inputs):
        # Reshape the weights
        reshaped_weights = self.reshape_weights(inputs)
        print(len(reshaped_weights))

        # Compress the weights using Huffman coding
        compressed_weights = []
        for i, weight_tensor in enumerate(reshaped_weights):
            weight_flattened = weight_tensor.flatten()
            weight_bytes = weight_flattened.tobytes()  # Convert to byte array
            encoder = dahuffman.HuffmanCodec.from_data(weight_bytes)
            self.codec.append(encoder) 
            compressed_data = encoder.encode(weight_bytes)  # Use encode() with byte array
            compressed_weights.append(compressed_data)
            print("Weight tensor shape:", weight_tensor.shape)
            print("Weight tensor size:", weight_tensor.size)
            print("Weight flattened size:", weight_flattened.size)
            print("Compressed data size:", len(compressed_data))
            
        return compressed_weights


    def decompressNN(self, compressed_weights):
        # Decompress the weights using Huffman coding
        decompressed_weights = []
        for i, compressed_data in enumerate(compressed_weights):
#             decoder = dahuffman.HuffmanCodec.from_data(compressed_data)
            decoder = self.codec[i]
            decompressed_data = decoder.decode(compressed_data)  # Use decode() directly
            
            print("Decompressed data size:", len(decompressed_data))
            weight_shape = self.net_model.get_weights()[0].shape  # Assuming all weight tensors have the same shape
            decompressed_size = np.prod(weight_shape) * np.dtype(np.float32).itemsize

            print("Expected decompressed size:", decompressed_size)
            print("Actual decompressed size:", len(decompressed_data))

            if len(decompressed_data) < decompressed_size:
                decompressed_data += b'\x00' * (decompressed_size - len(decompressed_data))

            elif len(decompressed_data) > decompressed_size:
                decompressed_data = decompressed_data[:decompressed_size]

            decompressed_array = np.frombuffer(bytes(decompressed_data), dtype=np.float32)
            decompressed_weights.append(decompressed_array.reshape(weight_shape))

        return decompressed_weights


    def call(self, inputs):
        return self.net_model(inputs)

    def reshape_weights(self, inputs):
        reshaped_weights = []
        current_index = 0
        for weight_tensor in self.net_model.get_weights():
            weight_shape = weight_tensor.shape
            weight_size = np.prod(weight_shape)
            # Reshape the weight tensor based on the size of the input array
            reshaped_weights.append(weight_tensor.reshape(weight_shape))
            current_index += weight_size
        return reshaped_weights
    
    def compare_weights(self, original_weights, decompressed_weights):
        differences = []
        for orig, decomp in zip(original_weights, decompressed_weights):
            orig_shape = orig.shape
            decomp_shape = decomp.shape
            decomp_reshaped = decomp.reshape(orig_shape) if orig_shape == decomp_shape else decomp
            diff = np.abs(orig - decomp_reshaped)
            differences.append(diff)
        return differences


    

model = get_model()
compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = random_state.rand(28, 28, 1)  # Generate random input data

# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
decompressed_weights = compNN.decompressNN(compressed_weights)

# print(compressed_weights)
# print(decompressed_weights)

# Compare the original weights with the decompressed weights
differences = compNN.compare_weights(model.get_weights(), decompressed_weights)
print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")
    
    
    

SyntaxError: invalid syntax (3522140960.py, line 1)

In [ ]:
import joblib

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.compressor = dahuffman.HuffmanCodec.from_data([])  # Initialize an empty compressor
        
    def compressNN(self, inputs):
        # Get the current weights of the network
        weights = self.net_model.get_weights()

        # Serialize the weights using joblib
        with open("weights.pkl", "wb") as file:
            joblib.dump(weights, file)

        # Read the serialized weights
        with open("weights.pkl", "rb") as file:
            serialized_weights = file.read()

        # Compress the serialized weights using Huffman encoding
        encoder = dahuffman.HuffmanCodec.from_data(serialized_weights)
        compressed_weights = encoder.encode(serialized_weights)

        return compressed_weights

    def decompressNN(self, compressed_weights):
        # Decode the compressed weights using Huffman decoding
        decoder = dahuffman.HuffmanCodec.from_dict(compressed_weights)
        serialized_weights = decoder.decode(compressed_weights)

        # Deserialize the weights using joblib
        with open("decompressed_weights.pkl", "wb") as file:
            file.write(serialized_weights)

        # Load the deserialized weights
        with open("decompressed_weights.pkl", "rb") as file:
            weights = joblib.load(file)

        # Set the decompressed weights to the network
        self.net_model.set_weights(weights)

        return self.net_model

In [3]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import CompressibleNN

kernel_initializer = 'he_normal'
activation = "relu"


def get_model(chs=128):
    shape = (28, 28, 1)

    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs * 2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model
    

model = get_model()

with open('original_model_weights.pkl', 'wb') as file:
    pickle.dump(model.get_weights(), file)
compNN = CompressibleNN(model)


# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = random_state.rand(28, 28, 1)  # Generate random input data

# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
# decompressed_weights = compNN.decompressNN(compressed_weights)

# print(compressed_weights)
# print(decompressed_weights)

# Compare the original weights with the decompressed weights
# differences = compNN.compare_weights(model.get_weights(), decompressed_weights)
# print("Differences between original and decompressed weights:")
# for i, diff in enumerate(differences):
#     print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")
    
    
    

Weight tensor shape: (784, 256)
Weight tensor size: 200704
Weight flattened size: 200704
Compressed data size: 739829
Weight tensor shape: (256,)
Weight tensor size: 256
Weight flattened size: 256
Compressed data size: 128
Weight tensor shape: (256, 128)
Weight tensor size: 32768
Weight flattened size: 32768
Compressed data size: 121256
Weight tensor shape: (128,)
Weight tensor size: 128
Weight flattened size: 128
Compressed data size: 64
Weight tensor shape: (128, 10)
Weight tensor size: 1280
Weight flattened size: 1280
Compressed data size: 4712
Weight tensor shape: (10,)
Weight tensor size: 10
Weight flattened size: 10
Compressed data size: 5


In [7]:
decompressed_weights = compNN.decompressNN(compressed_weights)

# print(compressed_weights)
# print(decompressed_weights)

# Compare the original weights with the decompressed weights
differences = compNN.compare_weights(model.get_weights(), decompressed_weights)

print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")

Decompressed data size: 802816
Expected decompressed size: 802816
Actual decompressed size: 802816
Decompressed data size: 1024
Expected decompressed size: 1024
Actual decompressed size: 1024
Decompressed data size: 131072
Expected decompressed size: 131072
Actual decompressed size: 131072
Decompressed data size: 512
Expected decompressed size: 512
Actual decompressed size: 512
Decompressed data size: 5120
Expected decompressed size: 5120
Actual decompressed size: 5120
Decompressed data size: 40
Expected decompressed size: 40
Actual decompressed size: 40
Differences between original and decompressed weights:
Layer 1: Max Difference = 0.0, Mean Difference = 0.0
Layer 2: Max Difference = 0.0, Mean Difference = 0.0
Layer 3: Max Difference = 0.0, Mean Difference = 0.0
Layer 4: Max Difference = 0.0, Mean Difference = 0.0
Layer 5: Max Difference = 0.0, Mean Difference = 0.0
Layer 6: Max Difference = 0.0, Mean Difference = 0.0


In [ ]:
print(model.get_weights())

[array([[-0.0387225 , -0.08137392, -0.02914773, ..., -0.00919168,
         0.02946452,  0.04639422],
       [-0.05397551, -0.00483679, -0.04930591, ...,  0.07424383,
         0.0271853 , -0.01563342],
       [-0.00271451, -0.0424049 , -0.00124385, ...,  0.00815904,
        -0.0186043 , -0.02493658],
       ...,
       [ 0.06878325,  0.03475486,  0.06290235, ...,  0.04817623,
        -0.08055975,  0.05437632],
       [-0.07352863,  0.01516827,  0.10736844, ..., -0.01694267,
         0.06866149,  0.00142415],
       [-0.03066814,  0.03886299,  0.03796422, ...,  0.08156086,
         0.02407725, -0.06137323]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
print(decompressed_weights)

[array([[-0.0387225 , -0.08137392, -0.02914773, ..., -0.00919168,
         0.02946452,  0.04639422],
       [-0.05397551, -0.00483679, -0.04930591, ...,  0.07424383,
         0.0271853 , -0.01563342],
       [-0.00271451, -0.0424049 , -0.00124385, ...,  0.00815904,
        -0.0186043 , -0.02493658],
       ...,
       [ 0.06878325,  0.03475486,  0.06290235, ...,  0.04817623,
        -0.08055975,  0.05437632],
       [-0.07352863,  0.01516827,  0.10736844, ..., -0.01694267,
         0.06866149,  0.00142415],
       [-0.03066814,  0.03886299,  0.03796422, ...,  0.08156086,
         0.02407725, -0.06137323]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
import joblib

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.compressor = dahuffman.HuffmanCodec.from_data([])  # Initialize an empty compressor
        
    def compressNN(self, inputs):
        # Get the current weights of the network
        weights = self.net_model.get_weights()

        # Serialize the weights using joblib
        with open("weights.pkl", "wb") as file:
            joblib.dump(weights, file)

        # Read the serialized weights
        with open("weights.pkl", "rb") as file:
            serialized_weights = file.read()

        # Compress the serialized weights using Huffman encoding
        encoder = dahuffman.HuffmanCodec.from_data(serialized_weights)
        compressed_weights = encoder.encode(serialized_weights)

        return compressed_weights

    def decompressNN(self, compressed_weights):
        # Decode the compressed weights using Huffman decoding
        decoder = dahuffman.HuffmanCodec.from_data(compressed_weights)
        serialized_weights = decoder.decode(compressed_weights)

        # Deserialize the weights using joblib
        with open("decompressed_weights.pkl", "wb") as file:
            file.write(serialized_weights)

        # Load the deserialized weights
        with open("decompressed_weights.pkl", "rb") as file:
            weights = joblib.load(file)

        # Set the decompressed weights to the network
        self.net_model.set_weights(weights)

        return self.net_model


# Instantiate the CompressibleNN class and create the original model
model = get_model()  # Replace with your own model creation code

# Save the original model's architecture
original_architecture = model.to_json()

compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = np.random.rand(10, 28, 28, 1)

# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
decompressed_weights = compNN.decompressNN(compressed_weights)

# Compare the original weights with the decompressed weights
# differences = compNN.compare_weights(original_weights, decompressed_weights)

# # Print the maximum difference for each weight matrix

KeyError: 51